<a href="https://colab.research.google.com/github/KasimbaJ/notes-app/blob/main/Housing_Prices_Anaysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from datetime import datetime
# from pandarallel import pandarallel
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from plotly.offline import iplot

In [5]:
hp = pd.read_csv("/content/drive/MyDrive/price_paid_records.csv")
hp.head()

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [6]:
hp.describe()

,Price
count,2.248935e+07
mean,1.782442e+05
std,3.903677e+05
min,1.000000e+00
25%,7.500000e+04
50%,1.300000e+05
75%,2.100000e+05
max,9.890000e+07


Describe price feature:

In [7]:
hp['Price'].describe()

count    2.248935e+07
mean     1.782442e+05
std      3.903677e+05
min      1.000000e+00
25%      7.500000e+04
50%      1.300000e+05
75%      2.100000e+05
max      9.890000e+07
Name: Price, dtype: float64

In [8]:
hp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 11 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   Transaction unique identifier      object
 1   Price                              int64 
 2   Date of Transfer                   object
 3   Property Type                      object
 4   Old/New                            object
 5   Duration                           object
 6   Town/City                          object
 7   District                           object
 8   County                             object
 9   PPDCategory Type                   object
 10  Record Status - monthly file only  object
dtypes: int64(1), object(10)
memory usage: 1.8+ GB


In [9]:
hp.count()

Transaction unique identifier        22489348
Price                                22489348
Date of Transfer                     22489348
Property Type                        22489348
Old/New                              22489348
Duration                             22489348
Town/City                            22489348
District                             22489348
County                               22489348
PPDCategory Type                     22489348
Record Status - monthly file only    22489348
dtype: int64

Printing features list


In [10]:
feat_cols = hp.columns
feat_cols

Index(['Transaction unique identifier', 'Price', 'Date of Transfer',
       'Property Type', 'Old/New', 'Duration', 'Town/City', 'District',
       'County', 'PPDCategory Type', 'Record Status - monthly file only'],
      dtype='object')

Clean the data: Check for null values

In [11]:
hp.isnull().sum()

Transaction unique identifier        0
Price                                0
Date of Transfer                     0
Property Type                        0
Old/New                              0
Duration                             0
Town/City                            0
District                             0
County                               0
PPDCategory Type                     0
Record Status - monthly file only    0
dtype: int64

Get rid of some features

In [12]:
hp.drop(columns = 'Transaction unique identifier', axis = 1, inplace = True)
hp.drop(columns = 'Duration', axis = 1, inplace = True)
hp.drop(columns = 'PPDCategory Type', axis = 1, inplace = True)
hp.drop(columns = 'Record Status - monthly file only', axis = 1, inplace = True)

hp.head()

,Price,Date of Transfer,Property Type,Old/New,Town/City,District,County
0,25000,1995-08-18 00:00,T,N,OLDHAM,OLDHAM,GREATER MANCHESTER
1,42500,1995-08-09 00:00,S,N,GRAYS,THURROCK,THURROCK
2,45000,1995-06-30 00:00,T,N,HIGHBRIDGE,SEDGEMOOR,SOMERSET
3,43150,1995-11-24 00:00,T,N,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE
4,18899,1995-06-23 00:00,S,N,WAKEFIELD,LEEDS,WEST YORKSHIRE


Analyse the data

In [13]:
print('Number of cities:', hp['Town/City'].nunique())

Number of cities: 1170


Recorded Transactions in London and York

In [22]:
print("Recorded transactions in London:", hp[hp['Town/City'] == 'LONDON']['Price'].count())
print("Recorded transactions in York:", hp[hp['Town/City'] == 'YORK']['Price'].count())
print("Recorded transactions in Burnley:", hp[hp['Town/City'] == 'BURNLEY']['Price'].count())

Recorded transactions in London: 1784194
Recorded transactions in York: 122206
Recorded transactions in Burnley: 47755


Create new dataset with the three cities

In [23]:
london_df = hp[hp['Town/City'] == 'LONDON'].copy()
york_df = hp[hp['Town/City'] == 'YORK'].copy()
burnley_df = hp[hp['Town/City'] == 'BURNLEY'].copy()

Merge the data

In [24]:
merged_cities_df = pd.concat([london_df, york_df, burnley_df])
merged_cities_df

,Price,Date of Transfer,Property Type,Old/New,Town/City,District,County
30,62000,1995-02-21 00:00,T,N,LONDON,BARNET,GREATER LONDON
37,77495,1995-01-05 00:00,T,N,LONDON,WALTHAM FOREST,GREATER LONDON
105,121250,1995-05-19 00:00,F,N,LONDON,BARNET,GREATER LONDON
107,128500,1995-03-01 00:00,T,N,LONDON,WANDSWORTH,GREATER LONDON
114,157500,1995-03-17 00:00,S,N,LONDON,ISLINGTON,GREATER LONDON
...,...,...,...,...,...,...,...
22488249,70000,2017-04-04 00:00,F,N,BURNLEY,BURNLEY,LANCASHIRE
22488250,70000,2017-04-04 00:00,F,N,BURNLEY,BURNLEY,LANCASHIRE
22488251,70000,2017-04-04 00:00,F,N,BURNLEY,BURNLEY,LANCASHIRE
22488252,70000,2017-04-04 00:00,F,N,BURNLEY,BURNLEY,LANCASHIRE
